In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import utils.helpers as helpers
import logging

In [11]:
boroughs = ['Bronx','Brooklyn', 'EWR', 'Manhattan', 'Queens', 'Staten Island']

In [12]:
import optuna
import warnings
warnings.filterwarnings('ignore')
from prophet import Prophet
import joblib

for b in boroughs:
    
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    file_handler = logging.FileHandler(f"logs/{b}_prophet_auto.log", mode='w')
    file_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    
    logger.info(f"Tuning for {b} - Prophet Model")

    df = pd.read_parquet(f'../data/{b} - all txns.parquet.gz')
    df = df[['num_txns_All']].reset_index()
    df['timestamp_hour'] = pd.to_datetime(df['timestamp_hour'])
    df = df.sort_values(by = 'timestamp_hour')
    df = df.rename(columns = {'num_txns_All': 'y', 'timestamp_hour': 'ds'})

    df_hist = df[df['ds'] < '2024-08-01']
    df_test = df[df['ds']>= '2024-08-01']


    # Objective function to minimize (MAE for Prophet)
    def objective(trial):
        # Sample hyperparameters from the Optuna trial using TPE
        changepoint_prior_scale = trial.suggest_uniform('changepoint_prior_scale', 0.001, 0.1)
        seasonality_prior_scale = trial.suggest_uniform('seasonality_prior_scale', 1.0, 10.0)
        holidays_prior_scale = trial.suggest_uniform('holidays_prior_scale', 1.0, 10.0)
        seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
        yearly_seasonality = trial.suggest_categorical('yearly_seasonality', [True, False])
        weekly_seasonality = trial.suggest_categorical('weekly_seasonality', [True, False])
        daily_seasonality = trial.suggest_categorical('daily_seasonality', [True, False])
        #fourier_order = trial.int_uniform('fourier_order', 3, 12)  # Fourier order for seasonality

        # Initialize Prophet with the sampled hyperparameters
        model = Prophet(
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            holidays_prior_scale=holidays_prior_scale,
            seasonality_mode=seasonality_mode,
            yearly_seasonality=yearly_seasonality,
            weekly_seasonality=weekly_seasonality,
            daily_seasonality=daily_seasonality
        )
        
        logger.info(f"""changepont_prior_scale: {changepoint_prior_scale}, seasonality_prior_scale: {seasonality_prior_scale}, holidays_prior_scale: {holidays_prior_scale}, seasonality_mode: {seasonality_mode}, 
        yearly_seasonality: {yearly_seasonality}, weekly_seasonality: {weekly_seasonality}, daily_seasonality: {daily_seasonality}""")

        # Add custom seasonality if necessary

        model.add_seasonality(name='hourly', period=24, fourier_order = 8)

        model.add_country_holidays(country_name='US')

        # Fit the model
        model.fit(df_hist)

        # Predict
        future = model.make_future_dataframe(periods=len(df_test), freq = 'H', include_history = False)
        forecast = model.predict(future)

        forecast = model.predict(future)

        # Calculate MAE or RMSE
        y_true = df_test['y'].values
        y_pred = forecast['yhat'][-len(df_test):].values
        results = helpers.performance_metrics(y_true, y_pred)
        
        logger.info(results)
        file_handler.flush()

        return results['rmse']
    
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())  # TPE is the default, but we explicitly specify it here
    study.optimize(objective, n_trials=50, show_progress_bar=True) 
    
    logger.info(f'\nHyperparameter Tuning Done for {b}')
    
    logger.info(f'Best Parameters: {study.best_params}')
    
    model = Prophet(**study.best_params)
    model.add_seasonality(name='hourly', period=24, fourier_order = 8)
    model.fit(df_hist)

    future = model.make_future_dataframe(periods=len(df_test), freq = 'H', include_history = False)
    forecast = model.predict(future)

    __ = helpers.performance_metrics(df_test['y'], forecast['yhat'])
    
    model_loc = f'../models/{b}_Prophet.joblib'
    forecast_loc = f'../data/forecasts/{b}_Aug2024_forecasts.csv'
    
    logger.info(f'Model Saved in {model_loc}')
    logger.info(f'Forecasts Saved in {forecast_loc}')
    
    file_handler.flush()
    
    joblib.dump(model, model_loc)
    forecast.to_csv(forecast_loc)
    

[I 2024-11-25 08:30:12,973] A new study created in memory with name: no-name-bb45d08a-e314-4761-9f64-0122d9e65b71


  0%|          | 0/50 [00:00<?, ?it/s]

08:30:18 - cmdstanpy - INFO - Chain [1] start processing
08:31:58 - cmdstanpy - INFO - Chain [1] done processing


MAE: 1492.44
MAPE: 18.84%
RMSE: 2027.07
R2: 0.75
[I 2024-11-25 08:31:59,679] Trial 0 finished with value: 2027.0743549952954 and parameters: {'changepoint_prior_scale': 0.08358982303180261, 'seasonality_prior_scale': 3.109267652554711, 'holidays_prior_scale': 6.759937632650955, 'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True}. Best is trial 0 with value: 2027.0743549952954.


08:32:04 - cmdstanpy - INFO - Chain [1] start processing
08:34:08 - cmdstanpy - INFO - Chain [1] done processing


...

MAE: 1493.50
MAPE: 18.87%
RMSE: 2027.55
R2: 0.75
[I 2024-11-25 12:00:54,479] Trial 48 finished with value: 2027.547179263431 and parameters: {'changepoint_prior_scale': 0.08956533709848997, 'seasonality_prior_scale': 8.266509845700837, 'holidays_prior_scale': 9.47830460732502, 'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True}. Best is trial 37 with value: 2026.216366855301.


12:01:01 - cmdstanpy - INFO - Chain [1] start processing
12:03:35 - cmdstanpy - INFO - Chain [1] done processing


MAE: 1492.67
MAPE: 18.85%
RMSE: 2027.09
R2: 0.75
[I 2024-11-25 12:03:35,652] Trial 49 finished with value: 2027.0876229128698 and parameters: {'changepoint_prior_scale': 0.09395973764427441, 'seasonality_prior_scale': 9.483337475924973, 'holidays_prior_scale': 7.283547152145196, 'seasonality_mode': 'additive', 'yearly_seasonality': True, 'weekly_seasonality': True, 'daily_seasonality': True}. Best is trial 37 with value: 2026.216366855301.


12:03:40 - cmdstanpy - INFO - Chain [1] start processing
12:05:06 - cmdstanpy - INFO - Chain [1] done processing


MAE: 1493.19
MAPE: 18.87%
RMSE: 2026.90
R2: 0.75
